In [1]:
import pandas as pd
import requests
import json
import re
from urllib.parse import quote

In [2]:
url = 'https://api.elsevier.com/content/search/scopus'
with open('config.json', 'r') as f:
    API_KEY = json.load(f)
query = '"inverse reinforcement learning"  AND  ( "system"  OR  "e-learning"  OR  "stochastic"  OR  "smart grids"  OR  "control"  OR  "system controller"  OR  "control tuning"  OR  "optimization")'

In [3]:
def query_to_scopus(url: str, query: str, api: str) -> list:
    return requests.get(url,
                        headers={'Accept': 'application/json', 'X-ELS-APIKey': api},
                        params={'query': query, 'view': 'COMPLETE', 'start': 0}).json()

class scopus_df:
    def __init__(self):
        self.columns = ['Authors', 'Title', 'Year', 'Cited By', 'Affiliations', 'Author Keywords', 'Source title']
        self.csv = pd.DataFrame(columns=self.columns)

    def get_authors(self, publication: dict) -> list:
        return [author['authname'] for author in publication['author']]

    def get_affiliations(self, publication: dict) -> list:
        return [aff['affilname'].replace(',', ';') for aff in publication['affiliation']]

    def append(self, publication: dict) -> None:
        authors = ','.join(get_authors(publication))
        title = publication['dc:title']
        year = re.findall(r'([\d]{4})', publication['prism:coverDisplayDate'])[0]
        source_title = publication['prism:publicationName']
        cites = publication['citedby-count']
        affiliations = ','.join(get_affiliations(publication))
        try:
            author_kw = ','.join([ii.lstrip().strip() for ii in publication['authkeywords'].split('|')])
        except KeyError as e:
            author_kw = ''
        self.csv = self.csv.append(pd.DataFrame([[authors, title, year, cites, affiliations, author_kw, source_title]], columns=self.columns), ignore_index=True)

In [4]:
query_parsed = f'TITLE-ABS-KEY({query})' # TODO: Save a history of all the querys with the number of results
api = API_KEY['api-key']
num_items = int(query_to_scopus(url, query_parsed, api)['search-results']['opensearch:totalResults'])

276

In [6]:
publications = scopus_df()
